# **차량 공유업체의 차량 파손 여부 분류하기**

## 0.미션

* 1) 미션1 : Data Preprocessing
    - **과제 수행 목표**
        - 본인의 구글 드라이브에 모델링 수행을 위해 적절한 폴더 및 파일로 **일관성 있게 정리**해야 합니다.
        - 제공된 데이터 : Car_Images.zip
            * Car_Images : 차량의 정상/파손 이미지 무작위 수집

* 2) 미션2 : CNN 모델링
    - **과제 수행 목표**
        - Tensorflow Keras를 이용하여 모델을 3개 이상 생성하세요.
            - 모델 구조와 파라미터는 자유롭게 구성하세요.
            - 단, 세부 목차에서 명시한 부분은 지켜주세요.

* 3) 미션3 : Data Argumentation & Transfer Learning
    - **과제 수행 목표**
        - 성능 개선을 위해 다음의 두가지를 시도하세요.
            * Data Augmentation을 적용하세요.(Image Generator)
            * Transfer Learning(VGG16)


## 1.환경설정 

### (1) 데이터셋 폴더 생성
- **세부요구사항**
    - C드라이브에 Datasets라는 폴더를 만드세요.
        - 구글드라이브를 사용하는경우 드라이브 첫 화면에 Datasets 라는 폴더를 만드세요. ('/content/drive/MyDrive/Datasets/')
    - 해당 폴더 안에 Car_Images.zip 파일을 넣으세요.

* 구글 Colab을 이용하는 경우

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 데이터셋 불러오기 
- **세부요구사항**
    - Car_Images.zip 파일을 C:/Datasets/ 경로에 압축 해제합니다.
    - zipfile 모듈을 이용하거나 다른 방식을 사용해도 됩니다.
        - 참고 자료 : [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 폴더구조(로컬)
        * C:/Datasets/ : 압축파일
        * C:/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 폴더구조(구글드라이브브)
        * /content/drive/MyDrive/Datasets/ : 압축파일
        * /content/drive/MyDrive/Datasets/Car_Images_train/ : 압축 해제한 이미지 저장소
    - 압축을 해제하면 다음과 같은 두 하위 폴더가 생성됩니다.
        * normal, abnormal : 각 폴더에는 이미지들이 있습니다.
        * 이후 단계에서 해당 경로로 부터 validation, test 셋을 추출하게 됩니다.
        

In [ ]:
import zipfile

In [ ]:
# 압축파일 경로
# 구글 드라이브인 경우 경로에 맞게 지정하세요.
dataset_path  = '/content/drive/MyDrive/Datasets/'
#dataset_path = 'C:/Datasets/'

file_path = dataset_path + 'Car_Images.zip'

In [ ]:
# 압축 해제
data = zipfile .ZipFile(file_path)
data.extractall('/content/drive/MyDrive/Datasets/Car_Images_train/')

KeyboardInterrupt: ignored

### (3) 이미지 저장을 위한 폴더 생성
- **세부요구사항**
    - train, validation, test 을 위해 각각 하위 폴더 normal과 abnormal를 준비합니다.
        - train
            * 정상 이미지 저장소 : C:/Datasets/Car_Images_train/normal/ 
                * 구글드라이브 :   /content/drive/MyDrive/Datasets/Car_Images_train/normal/
            * 파손 이미지 저장소 : C:/Datasets/Car_Images_train/abnormal/
                * 구글드라이브 : /content/drive/MyDrive/Datasets/Car_Images_train/abnormal/
        - val, test 역시 동일한 구조로 생성합니다.
    - 직접 탐색기에서 폴더를 생성할 수도 있고, os 모듈을 이용하여 코드로 작성할 수도 있습니다.
        - 참고 자료 : [os document](https://docs.python.org/3/library/os.html)

In [ ]:

# 각각 경로 지정
tr_n_path = '/content/drive/MyDrive/Datasets/Car_Images_train/normal/'
tr_ab_path = '/content/drive/MyDrive/Datasets/Car_Images_train/abnormal/'

# train 폴더는 압축을 해제하면서 이미 생성 되어 있습니다.

# test 폴더 만들기 os.mkdir()
!mkdir /content/drive/MyDrive/Datasets/Car_Images_test


# validation 폴더 만들기
!mkdir /content/drive/MyDrive/Datasets/Car_Images_val



mkdir: cannot create directory ‘/content/drive/MyDrive/Datasets/Car_Images_test’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Datasets/Car_Images_val’: File exists


## 2.데이터 전처리

### (1) 데이터 분할 : Training set | Validation set | Test set 생성
- **세부요구사항**
    - Training set, Validation set, Test set을 만듭니다.
        * size
            * test : 전체에서 20%를 추출합니다.
            * validation : test를 떼어낸 나머지에서 다시 20%를 추출합니다.
        * 데이터는 랜덤하게 추출해야 합니다.
            - random, shutil 모듈을 이용하여 랜덤하게 추출할 수 있습니다.
                - [random document](https://docs.python.org/3/library/random.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
            * 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

#### 1) test, validation 크기를 지정

In [ ]:
import random, shutil
import os

In [ ]:
# 전체 이미지 갯수를 확인합니다.
len(os.listdir(tr_n_path)) , len(os.listdir(tr_ab_path))

(302, 303)

In [ ]:
# test 사이즈 : 전체 이미지의 20%
te_data_num = [round(len(os.listdir(tr_n_path))*0.2), round(len(os.listdir(tr_ab_path))*0.2)]
print(te_data_num)

# validation 사이즈 : test를 제외한 나머지 중에서 20%
val_data_num = [ round((len(os.listdir(tr_n_path))-te_data_num[0])*0.2) , round((len(os.listdir(tr_ab_path))-te_data_num[1])*0.2) ]
print(val_data_num)

# train 사이즈
train_data_num = [len(os.listdir(tr_n_path)) - te_data_num[0] - val_data_num[0],
                  len(os.listdir(tr_ab_path))- te_data_num[1] - val_data_num[1]]
print(train_data_num)

[60, 61]
[48, 48]
[194, 194]


#### 2) test 셋 추출

In [ ]:
#random.seed(2023)
!pip install split-folders
import splitfolders
data = '/content/drive/MyDrive/Datasets/Car_Images_train'
splitfolders.ratio(data, output='/content/drive/MyDrive/Datasets/', seed=2023, ratio=(0.7, 0.1, 0.2))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 605 files [02:36,  3.87 files/s]


In [ ]:
# 추출 후 이미지 갯수 확인
test = '/content/drive/MyDrive/Datasets/test/normal'
train = '/content/drive/MyDrive/Datasets/train'
val = '/content/drive/MyDrive/Datasets/val'
print(len(os.listdir('/content/drive/MyDrive/Datasets/train/abnormal')))

242


#### 3) validation 셋 추출

In [ ]:
# random.seed(2023)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2023)

In [ ]:
# 추출 후 이미지 갯수 확인




### (2) 데이터 복사 및 이동
- **세부요구사항**
    - 분할된 데이터를 복사 이동합니다.
        - 새로운 폴더에 저장하는 데이터로 "3.모델링I"에서 사용합니다.
        - 기존 폴더는 "4.모델링II > (1) Data Augmentation"에서 사용합니다.
    - Training set | Validation set | Test set의 데이터를 **새로운 폴더**에 복사하세요.
        - 새로운 폴더 명
            * copy_images/trainset
            * copy_images/validset
            * copy_images/testset
        - 새로운 폴더에는 normal, abnormal 파일 모두를 복사합니다. 
            * 파일을 구분하기 위해 abnormal 파일들은 파일명 앞에 접두사 'ab_'를 붙입시다.
        - os, shutil 모듈을 활용하세요.

#### 1) abnormal 파일 복사

* 복사하기 : shutil.copytree()

In [ ]:
shutil.copytree('/content/drive/MyDrive/Datasets/test/abnormal', '/content/drive/MyDrive/Datasets/copy_images/testset')
shutil.copytree('/content/drive/MyDrive/Datasets/val/abnormal', '/content/drive/MyDrive/Datasets/copy_images/validset')
shutil.copytree('/content/drive/MyDrive/Datasets/train/abnormal', '/content/drive/MyDrive/Datasets/copy_images/trainset')

'/content/drive/MyDrive/Datasets/copy_images/trainset'

In [ ]:
# import shutil
# shutil.rmtree('/content/drive/MyDrive/Datasets/copy_images')

In [ ]:
import glob


* abnormal 이미지 이름의 접두어 "ab_" 붙이기 : os.rename

In [ ]:
import glob

path = '/content/drive/MyDrive/Datasets/copy_images/testset'
files = glob.glob(path + '/*')

for f in files:
    os.rename(f, os.path.join(path, 'ab_' + os.path.basename(f)))

In [ ]:
path = '/content/drive/MyDrive/Datasets/copy_images/validset'
files = glob.glob(path + '/*')

for f in files:
    os.rename(f, os.path.join(path, 'ab_' + os.path.basename(f)))

In [ ]:
path = '/content/drive/MyDrive/Datasets/copy_images/trainset'
files = glob.glob(path + '/*')

for f in files:
    os.rename(f, os.path.join(path, 'ab_' + os.path.basename(f)))

#### 2) normal 파일 복사

In [ ]:
from distutils.dir_util import copy_tree
copy_tree('/content/drive/MyDrive/Datasets/test/normal', '/content/drive/MyDrive/Datasets/copy_images/testset')
copy_tree('/content/drive/MyDrive/Datasets/val/normal', '/content/drive/MyDrive/Datasets/copy_images/validset')
copy_tree('/content/drive/MyDrive/Datasets/train/normal', '/content/drive/MyDrive/Datasets/copy_images/trainset')

['/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 00.18.21 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 01.00.12 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 14.36.16 - photo of part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-10 23.42.04 - photo of a part of car without blemish.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 14.28.22 - part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 01.04.23 - photo of a part of car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 01.35.30 - photo of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DALLíñE 2023-03-11 14.43.32 - photo of part of a car.png',
 '/content/drive/MyDrive/Datasets/copy_images/trainset/DA

* 데이터 갯수 조회

In [ ]:
print(len(os.listdir(dataset_path+'copy_images/trainset/')))
print(len(os.listdir(dataset_path+'copy_images/validset/')))
print(len(os.listdir(dataset_path+'copy_images/testset/')))

483
151
123


## 3.모델링 I
* **세부요구사항**
    * 모델링을 위한 데이터 구조 만들기
        * x : 이미지를 array로 변환합니다.
        * y : 이미지 갯수만큼 normal - 0, abnormal - 1 로 array를 만듭니다.
    * 모델을 최소 3개 이상 만들고 성능을 비교합니다.
        * 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
        * 전처리 과정에서 생성한 Validation set을 적절하게 사용하세요.
        * Early Stopping을 반드시 사용하세요.
            * 최적의 가중치를 모델에 적용하세요.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix

### (1) X : image to array
- **세부요구사항**
    * 모델링을 위해서는 np.array 형태로 데이터셋을 만들어야 합니다.
    * Training set / Validation set / Test set의 X는 이미지 형태로 되어있습니다. 
    * 이미지 파일을 불러와 train, valid, test 각각 array 형태로 변환해 봅시다.
        * 각 폴더로 부터 이미지 목록을 만들고
        * 이미지 한장씩 적절한 크기로 로딩하여 (keras.utils.load_img)
            * 이미지가 너무 크면 학습시간이 많이 걸리고, 메모리 부족현상이 발생될 수 있습니다.
            * 이미지 크기를 280 * 280 * 3 이내의 크기를 설정하여 로딩하시오.
            * array로 변환 (keras.utils.img_to_array, np.expand_dims)
        * 데이터셋에 추가합니다.(데이터셋도 array)

# #### 1) 이미지 목록 만들기
* train, validation, test 폴더로 부터 이미지 목록을 생성합니다.

In [ ]:
# 이미지 목록 저장
img_train_list = os.listdir(dataset_path+'copy_images/trainset/')
img_valid_list = os.listdir(dataset_path+'copy_images/validset/')
img_test_list = os.listdir(dataset_path+'copy_images/testset/')

In [ ]:
# 메모리, 처리시간을 위해서 이미지 크기 조정
img_size = 280 ## 사이즈 조정 가능

#### 2) 이미지들을 배열 데이터셋으로 만들기

In [ ]:
from tensorflow.keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input


files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')

testset = []

for path in files :
    img = image.load_img(path, grayscale = False, target_size = (280,280) )
    img = image.img_to_array(img)
    img = preprocess_input(img)
    testset.append(img)

    
testset = np.array(testset) 


In [ ]:
files2 = glob.glob('/content/drive/MyDrive/Datasets/copy_images/trainset/*')
trainset = []
for path in files2 :
    img = image.load_img(path, grayscale = False, target_size = (280,280) )
    img = image.img_to_array(img)
    img = preprocess_input(img)
    trainset.append(img)
trainset = np.array(trainset)    


In [ ]:
files3 = glob.glob('/content/drive/MyDrive/Datasets/copy_images/validset/*')
validset = []

for path in files3 :
    img = image.load_img(path, grayscale = False, target_size = (280,280) )
    img = image.img_to_array(img)
    img = preprocess_input(img)
    validset.append(img)
validset = np.array(validset)   

In [ ]:
testset.shape, trainset.shape, validset.shape

((123, 280, 280, 3), (483, 280, 280, 3), (151, 280, 280, 3))

In [ ]:
#x scailing

max_n, min_n = trainset.max(), trainset.min()
x_train = (trainset - min_n) / (max_n - min_n)
x_val = (validset - min_n) / (max_n - min_n)
x_test = (testset - min_n) / (max_n - min_n)

### (2) y : 클래스 만들기
- **세부요구사항**
    - Training set / Validation set / Test set의 y를 생성합니다.
        - 각각 normal, abnormal 데이터의 갯수를 다시 확인하고
        - normal을 0, abnormal을 1로 지정합니다.

In [ ]:
# 데이터 갯수 확인
print( len(img_train_list) )
print( len([val for val in img_train_list if val.startswith('ab_')]) )
print('---')
print( len(img_valid_list) )
print( len([val for val in img_valid_list if val.startswith('ab_')]) )
print('---')
print( len(img_test_list) )
print( len([val for val in img_test_list if val.startswith('ab_')]) )

483
242
---
151
76
---
123
62


* y_train, y_valid, y_test 만들기
    * normal, abnormal 데이터의 갯수를 다시 확인하고 normal을 0, abnormal을 1로 지정합니다.

In [ ]:
y_test = []
# files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')

for path in img_test_list:
    if path.startswith("ab_"):
        y_test.append(1)
    else:
        y_test.append(0)
y_test = np.array(y_test)   
# label_v = len(np.unique(names))
# label_v
#test, train, val

In [ ]:
y_train = []
# files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')

for path in img_train_list:
    if path.startswith("ab_"):
        y_train.append(1)
    else:
        y_train.append(0)
y_train = np.array(y_train)

In [ ]:
y_val = []
# files = glob.glob('/content/drive/MyDrive/Datasets/copy_images/testset/*')

for path in img_valid_list:
    if path.startswith("ab_"):
        y_val.append(1)
    else:
        y_val.append(0)
y_val = np.array(y_val)

In [ ]:
y_test.shape, y_train.shape, y_val.shape

((123,), (483,), (151,))

In [ ]:
from tensorflow.keras.utils import to_categorical

label_v = len(np.unique(y_test))
y_test = to_categorical(y_test, label_v)

In [ ]:
label_v = len(np.unique(y_train))
y_train = to_categorical(y_train, label_v)

In [ ]:
label_v = len(np.unique(y_val))
y_val = to_categorical(y_val, label_v)

In [ ]:
testset.shape, trainset.shape, validset.shape

((123, 280, 280, 3), (483, 280, 280, 3), (151, 280, 280, 3))

In [ ]:
x_test.shape, x_train.shape, x_val.shape

((123, 280, 280, 3), (483, 280, 280, 3), (151, 280, 280, 3))

In [ ]:
y_test.shape, y_train.shape, y_val.shape

((123, 2), (483, 2), (151, 2))

In [ ]:
testset.shape

AttributeError: ignored

### (3) 모델1
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout, AveragePooling2D

In [ ]:
# # 혹시 이미 그려둔 그래프가 있다면 날려줘!
clear_session()

# model에 순차적으로 레이어를 쌓아가겠다는 의도!
model = keras.models.Sequential()

model.add( Input(shape=(280,280,3)) )
model.add( Conv2D(filters=32,
                  kernel_size=(3, 3),
                  activation='relu') )
model.add( Conv2D(filters=32,
                  kernel_size=(3,3),
                  activation='relu') )
model.add( MaxPool2D((2,2)) )
model.add( Dropout(0.20) )

model.add( Conv2D(filters=16,
                  kernel_size=(3, 3),
                  activation='relu',
                  padding='same') )
model.add( Conv2D(filters=16,
                  kernel_size=(3, 3),
                  activation='relu',
                  padding='same') )
model.add( MaxPool2D((2,2)) )
model.add( Dropout(0.25) )


model.add( Flatten() )
model.add( Dense(10, activation = 'relu') )
model.add( BatchNormalization()  )
model.add( Dropout(0.25) )
model.add( Dense(2, activation = 'softmax') )

# 모델 구조 확인
model.summary()

# keras.backend.clear_session()

# il = Input(shape=(img_size, img_size, 3))
# cl1 = Conv2D(filters = 32, kernel_size = (8, 8), padding = 'same', activation = 'relu')(il)
# bl1 = BatchNormalization()(cl1)
# ml1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(bl1)
# dl1 = Dropout(0.25)(ml1)

# fl = Flatten()(dl1)
# hl = Dense(256, activation='relu')(fl)
# bl3 = BatchNormalization()(hl)
# dl3 = Dropout(0.35)(bl3)
# ol = Dense(2, activation='sigmoid')(dl3)

# model = keras.models.Model(il, ol)

# model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')

# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 278, 278, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 276, 276, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 138, 138, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 138, 138, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 138, 138, 16)      4624      
                                                                 
 conv2d_3 (Conv2D)           (None, 138, 138, 16)      2320      
                                                        

In [ ]:
# 모델 컴파일

model.compile(loss = keras.losses.categorical_crossentropy, metrics=['accuracy'],
              optimizer = 'adam')

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
es = EarlyStopping(monitor='val_loss', # 개선 대상
                   min_delta=0,        # 개선되고 있다고 판단하기 위한 최소 변화량
                   patience=5,         # 개선없는 epoch를 얼마나 기다려줄거야?
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
mcp = ModelCheckpoint(filepath='/content/model_save.h5',  # 모델 저장 경로
                      monitor='val_loss',                 # 모델 저장의 관심 대상
                      verbose=1,                          # 어느 시점에서 저장되는지 알려줌
                      save_best_only=True,                # 최고 성능 모델만 저장
                      save_weights_only=False             # 가중치만 저장할 것인지, 모델 구조도 저장할 것인지
                      )

In [ ]:
history = model.fit(trainset, y_train, 
                    epochs=10000, validation_data=(validset, y_val),
                    verbose=1, callbacks=[es, mcp]) 

Epoch 1/10000
16/16 [==============================] - ETA: 0s - loss: 0.5209 - accuracy: 0.7764
Epoch 1: val_loss improved from inf to 1.69071, saving model to /content/model_save.h5
16/16 [==============================] - 158s 10s/step - loss: 0.5209 - accuracy: 0.7764 - val_loss: 1.6907 - val_accuracy: 0.5099
Epoch 2/10000
16/16 [==============================] - ETA: 0s - loss: 0.4169 - accuracy: 0.8551 
Epoch 2: val_loss improved from 1.69071 to 1.16796, saving model to /content/model_save.h5
16/16 [==============================] - 161s 10s/step - loss: 0.4169 - accuracy: 0.8551 - val_loss: 1.1680 - val_accuracy: 0.6623
Epoch 3/10000
16/16 [==============================] - ETA: 0s - loss: 0.3911 - accuracy: 0.8592
Epoch 3: val_loss improved from 1.16796 to 0.66334, saving model to /content/model_save.h5
16/16 [==============================] - 154s 10s/step - loss: 0.3911 - accuracy: 0.8592 - val_loss: 0.6633 - val_accuracy: 0.7483
Epoch 4/10000
16/16 [=========================

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model.predict(testset)

4/4 [==============================] - 8s 2s/step


In [ ]:
# y_test = np.argmax(y_test, axis = 1)
# y_test

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1])

In [ ]:
print(confusion_matrix (y_test, y_pred))
print(classification_report (y_test, y_pred))

[[53  8]
 [18 44]]
              precision    recall  f1-score   support

           0       0.75      0.87      0.80        61
           1       0.85      0.71      0.77        62

    accuracy                           0.79       123
   macro avg       0.80      0.79      0.79       123
weighted avg       0.80      0.79      0.79       123



### (4) 모델2
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential()

model.add(Input(shape=(280,280,3)))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(AveragePooling2D(strides=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 280, 280, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 280, 280, 64)      36928     
                                                                 
 batch_normalization (BatchN  (None, 280, 280, 64)     256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 140, 140, 64)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 140, 140, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 140, 140, 64)      3

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

In [ ]:
es = EarlyStopping(monitor='val_loss', # 개선 대상
                   min_delta=0,        # 개선되고 있다고 판단하기 위한 최소 변화량
                   patience=15,         # 개선없는 epoch를 얼마나 기다려줄거야?
                   verbose=1,
                   restore_best_weights=True)

In [ ]:
history = model.fit(trainset, y_train, 
                    epochs=10000, validation_data=(validset, y_val),
                    verbose=1, callbacks=[es]) 

Epoch 1/10000
16/16 [==============================] - 9s 564ms/step - loss: 0.0761 - accuracy: 0.9793 - val_loss: 0.4068 - val_accuracy: 0.7881
Epoch 2/10000
16/16 [==============================] - 8s 532ms/step - loss: 0.0601 - accuracy: 0.9896 - val_loss: 0.3594 - val_accuracy: 0.8808
Epoch 3/10000
16/16 [==============================] - 8s 529ms/step - loss: 0.1255 - accuracy: 0.9607 - val_loss: 0.5584 - val_accuracy: 0.7550
Epoch 4/10000
16/16 [==============================] - 8s 505ms/step - loss: 0.1249 - accuracy: 0.9627 - val_loss: 0.7583 - val_accuracy: 0.7550
Epoch 5/10000
16/16 [==============================] - 8s 513ms/step - loss: 0.0918 - accuracy: 0.9793 - val_loss: 0.3676 - val_accuracy: 0.8808
Epoch 6/10000
16/16 [==============================] - 8s 517ms/step - loss: 0.0631 - accuracy: 0.9876 - val_loss: 0.3583 - val_accuracy: 0.8742
Epoch 7/10000
16/16 [==============================] - 8s 502ms/step - loss: 0.0732 - accuracy: 0.9855 - val_loss: 0.3448 - val_ac

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=10000, validation_data=(x_val, y_val),
                    verbose=1, callbacks=[es]) 

Epoch 1/10000
16/16 [==============================] - 18s 778ms/step - loss: 0.6183 - accuracy: 0.7557 - val_loss: 0.9345 - val_accuracy: 0.4305
Epoch 2/10000
16/16 [==============================] - 8s 528ms/step - loss: 0.4443 - accuracy: 0.8323 - val_loss: 0.9738 - val_accuracy: 0.5033
Epoch 3/10000
16/16 [==============================] - 8s 522ms/step - loss: 0.3668 - accuracy: 0.8571 - val_loss: 0.9466 - val_accuracy: 0.5166
Epoch 4/10000
16/16 [==============================] - 8s 501ms/step - loss: 0.3303 - accuracy: 0.8758 - val_loss: 1.1358 - val_accuracy: 0.5033
Epoch 5/10000
16/16 [==============================] - 8s 500ms/step - loss: 0.2860 - accuracy: 0.8923 - val_loss: 1.3250 - val_accuracy: 0.5033
Epoch 6/10000
16/16 [==============================] - 8s 496ms/step - loss: 0.2632 - accuracy: 0.9213 - val_loss: 1.3747 - val_accuracy: 0.5033
Epoch 7/10000
16/16 [==============================] - 8s 517ms/step - loss: 0.2014 - accuracy: 0.9379 - val_loss: 1.4721 - val_a

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred = model.predict(testset)

4/4 [==============================] - 0s 99ms/step


In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [ ]:
# y_test = np.argmax(y_test, axis=1)
# y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
print(confusion_matrix (y_test, y_pred))
print(classification_report (y_test, y_pred))

[[46 15]
 [12 50]]
              precision    recall  f1-score   support

           0       0.79      0.75      0.77        61
           1       0.77      0.81      0.79        62

    accuracy                           0.78       123
   macro avg       0.78      0.78      0.78       123
weighted avg       0.78      0.78      0.78       123



In [ ]:
y_pred = model.predict(testset)
y_pred = np.argmax(y_pred, axis=1)
history = model.fit(trainset, y_train, 
                    epochs=10000, validation_data=(validset, y_val),
                    verbose=1, callbacks=[es]
print(classification_report (y_test, y_pred))

### (5) 모델3
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 validation_data로 validation set을 사용하시오.
    - 반드시 Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

#### 1) 구조 설계

#### 2) 학습
* EarlyStopping 설정하고 학습시키기

#### 3) test set으로 예측하고 평가하기
* 평가는 confusion_matrix, classification_report 활용

## 4.모델링 II
* **세부요구사항**
    - 성능을 높이기 위해서 다음의 두가지를 시도해 봅시다.
        - Data Augmentation을 통해 데이터를 증가 시킵니다.
            - ImageDataGenerator를 사용합니다.
        - 사전 학습된 모델(Transfer Learning)을 가져다 사용해 봅시다.
            - VGG16(이미지넷)을 사용해 봅시다.

### (1) Data Augmentation
- **세부요구사항**
    * 모델 학습에 이용할 이미지 데이터를 증강시키세요.
    * Keras의 ImageDataGenerator를 이용
        - [ImageDataGenerator document](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

    * image generator를 이용하여 학습
        * 모델 구조는 이미 생성한 1,2,3 중 하나를 선택하여 학습


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 280 ## 사이즈 조정 가능

train_path = dataset_path+'train/'
valid_path = dataset_path+'val/'

#### 1) ImageGenerator 생성
* ImageDataGenerator 함수 사용
    * 주요 옵션
        * rotation_range: 무작위 회전을 적용할 각도 범위
        * zoom_range: 무작위 줌을 적용할 범위 [1-zoom_range, 1+zoom_range]
        * horizontal_flip: 무작위 좌우반전을 적용할지 여부
        * vertical_flip: 무작위 상하반전을 적용할지 여부
        * rescale: 텐서의 모든 값을 rescale 값으로 나누어줌 (이 경우에는 255로 나누어서 0~1사이의 값으로 변경)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10,      # 이미지 회전 -30 부터 30까지
                         width_shift_range=0.3,  # 이미지 좌우 이동 
                         height_shift_range=0.3, # 이미지 상하 이동
                         zoom_range=0.3,         # 확대/축소 범위
                         shear_range=0.1,        # 비스듬히 늘림
                         horizontal_flip=True,   # 가로 전환
                         vertical_flip=True,     # 세로 전환           
                         fill_mode='nearest'
                         )    # 마지막 옵션 주의하자. 이미지 회전, 이동, 축소할 때 발생하는 공간을 채우는 방식

train_datagen.fit(x_train)
valid_datagen = ImageDataGenerator(rotation_range=10,      # 이미지 회전 -30 부터 30까지
                         width_shift_range=0.3,  # 이미지 좌우 이동 
                         height_shift_range=0.3, # 이미지 상하 이동
                         zoom_range=0.3,         # 확대/축소 범위
                         shear_range=0.1,        # 비스듬히 늘림
                         horizontal_flip=True,   # 가로 전환
                         vertical_flip=True,     # 세로 전환           
                         fill_mode='nearest')    # 마지막 옵션 주의하자. 이미지 회전, 이동, 축소할 때 발생하는 공간을 채우는 방식
valid_datagen.fit(x_val)


#### 2) 경로로 부터 이미지 불러 올 준비
* .flow_from_directory 이용
    * 디렉토리에서 이미지를 가져와서 데이터 증강을 적용하고 batch 단위로 제공하는 generator를 생성합니다.
    * 이미지를 불러올 때 target_size로 크기를 맞추고, 
    * class_mode로 이진 분류(binary)를 수행하도록 지정합니다.


In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (280, 280),
    batch_size = 64,
    class_mode = 'binary',
    # calsses = ['normal', 'abnomal'])

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size = (280, 280),
    batch_size = 64,
    class_mode = 'binary')


SyntaxError: ignored

#### 3) 학습
- **세부요구사항**
    - Conv2D, MaxPooling2D, Flatten, Dense 레이어들을 이용하여 모델을 설계
    - 학습시 train_generator 이용. 
    - validation_data = valid_generator 지정
    - Early Stopping 적용
    - 평가시, confusion matrix, accuracy, recall, precision, f1 score 등을 이용하시오.

* 구조 설계

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import Conv2D, GlobalAveragePooling2D,Dense
keras.backend.clear_session()

base_model = InceptionV3(weights='imagenet',       # 이미지넷 데이터 바탕으로 미리 학습된 것
                         include_top=False,        # 마지막 레이어 빼고!
                         input_shape= (280,280,3)) # 입력 데이터의 형태

new_output = GlobalAveragePooling2D()(base_model.output)
new_output = Dense(1,activation = 'sigmoid')(new_output)

model = keras.models.Model(base_model.inputs, new_output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 139, 139, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 139, 139, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import Conv2D, GlobalAveragePooling2D,Dense
keras.backend.clear_session()

base_model = InceptionV3(weights='imagenet',       # 이미지넷 데이터 바탕으로 미리 학습된 것
                         include_top=False,        # 마지막 레이어 빼고!
                         input_shape= (280,280,3)) # 입력 데이터의 형태

new_output = GlobalAveragePooling2D()(base_model.output)
new_output = Dense(2,activation = 'softmax')(new_output)

model = keras.models.Model(base_model.inputs, new_output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 280, 280, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 139, 139, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 139, 139, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

* 학습
    * EarlyStopping 설정하기
    * 학습 데이터에 train_generator, validation_data=valid_generator 사용

In [ ]:
#사용할 레이어 수준 수정
for idx, layer in enumerate(model.layers) :
    if idx < 213 :
        layer.trainable = False
    else :
        layer.trainable = True#마지막 100개에 대해서만 튜닝할 것 이다!

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',         # 관측 대상
                   min_delta=0,                # 임계값
                   patience=5,
                   verbose=1,                  # 임계값 이상으로 변하지 않을때 몇 번 더 지켜볼래
                   restore_best_weights=True)  # 최적의 epoch 가중치를 적용!

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

In [ ]:
model.fit(train_generator,epochs=1000, validation_data=valid_generator,
                 verbose=1, callbacks=[es])

Epoch 1/1000
8/8 [==============================] - 51s 5s/step - loss: 0.6743 - accuracy: 0.7060 - val_loss: 7.1775 - val_accuracy: 0.7020
Epoch 2/1000
8/8 [==============================] - 34s 4s/step - loss: 0.4665 - accuracy: 0.7971 - val_loss: 1105.7473 - val_accuracy: 0.4967
Epoch 3/1000
8/8 [==============================] - 36s 5s/step - loss: 0.4559 - accuracy: 0.7909 - val_loss: 311.1071 - val_accuracy: 0.4967
Epoch 4/1000
8/8 [==============================] - 33s 4s/step - loss: 0.4621 - accuracy: 0.7847 - val_loss: 42.6003 - val_accuracy: 0.4967
Epoch 5/1000
8/8 [==============================] - 34s 4s/step - loss: 0.4501 - accuracy: 0.7888 - val_loss: 175.2339 - val_accuracy: 0.4967
Epoch 6/1000
8/8 [==============================] - 36s 5s/step - loss: 0.4323 - accuracy: 0.8157 - val_loss: 134.0241 - val_accuracy: 0.4967
Epoch 6: early stopping


In [ ]:
y_pred=model.predict(x_test)

4/4 [==============================] - 2s 121ms/step


#### 4) 성능 평가
* 평가는 confusion_matrix, classification_report 활용

In [ ]:
y_pred=y_pred.argmax(axis=1)

In [ ]:
y_test=y_test.argmax(axis=1)

AxisError: ignored

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
print (f'정확도는 {accuracy_score(y_test,y_pred)}입니다.')
print (confusion_matrix(y_test,y_pred))
print (classification_report(y_test,y_pred))

ValueError: ignored

In [ ]:
'''
성능 확인을 위해
Ctrl+Enter를 이용하여
반복 실행 해보자!
'''
import matplotlib.pyplot as plt

import random as rd
id = rd.randrange(0,121)

print(f'id = {id}')
print(f'다음 그림은  {y_test[id]} 입니다.')
print(f'모델의 예측 : {y_pred[id]}')


if y_test[id] == y_pred[id] :
    print('정답입니다')
else : 
    print('틀렸어요')
plt.figure(figsize=(5,5))    
plt.imshow(x_test[id])
plt.show()

### (2) Transfer Learning
- **세부요구사항**
    * VGG16 모델은 1000개의 클래스를 분류하는 데 사용된 ImageNet 데이터셋을 기반으로 사전 학습된 가중치를 가지고 있습니다. 
        * 따라서 이 모델은 이미지 분류 문제에 대한 높은 성능을 보입니다.
        * 이 모델은 보통 전이학습(transfer learning)에서 기본적으로 사용되며, 특히 대규모 데이터셋이 없을 때는 기본 모델로 사용되어 fine-tuning을 수행합니다.
    * VGG16 함수로 부터 base_model 저장


In [ ]:
from tensorflow.keras.applications import VGG16

#### 1) VGG16 불러와서 저장하기
* include_top=False로 설정하여 분류기를 제외하고 미리 학습된 가중치 imagenet을 로드합니다.
* .trainable을 True로 설정하여 모델의 모든 레이어들이 fine-tuning에 대해 업데이트되도록 합니다.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(280, 280, 3))
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 280, 280, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 280, 280, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 280, 280, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 140, 140, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 140, 140, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 140, 140, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 70, 70, 128)       0     

#### 2) V1GG6과 연결한 구조 설계
* VGG16을 불러와서 Flatten, Dense 등으로 레이어 연결하기

In [ ]:
from tensorflow.keras.models import Sequential

additional_model = keras.models.Sequential()
additional_model.add(base_model)
additional_model.add(keras.layers.Flatten())
additional_model.add(keras.layers.Dense(4096, activation='relu'))
additional_model.add(keras.layers.Dense(2048, activation='relu'))
additional_model.add(keras.layers.Dense(1024, activation='relu'))
additional_model.add(keras.layers.Dense(1, activation='softmax'))
 
 
additional_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_5 (Dense)             (None, 4096)              134221824 
                                                                 
 dense_6 (Dense)             (None, 2048)              8390656   
                                                                 
 dense_7 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_8 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 159,427,394
Trainable params: 144,712,70

#### 3) 학습
- **세부요구사항**
    - 모델 학습 과정에 알맞은 보조 지표를 사용하세요.
    - 데이터
        * Image Generator를 연결하거나
        * 기존 train, validation 셋을 이용해도 됩니다.
        - Early Stopping을 반드시 사용하세요.
        - 최적의 가중치를 모델에 적용하세요.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss',         # 관측 대상
                   min_delta=0,                # 임계값
                   patience=5,
                   verbose=1,                  # 임계값 이상으로 변하지 않을때 몇 번 더 지켜볼래
                   restore_best_weights=True)  # 최적의 epoch 가중치를 적용!

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
             optimizer=keras.optimizers.Adam(learning_rate=0.001) )

In [ ]:
history = model.fit(x_train, y_train, 
                    epochs=10000, validation_data=(x_val, y_val),
                    verbose=1, callbacks=[es]) 

Epoch 1/10000


ValueError: ignored

In [ ]:
model.fit(train_generator,epochs=1000, validation_data=valid_generator,
                 verbose=1, callbacks=[es])

Epoch 1/1000
8/8 [==============================] - 74s 5s/step - loss: 0.3949 - accuracy: 0.8116 - val_loss: 1.2542 - val_accuracy: 0.5166
Epoch 2/1000
8/8 [==============================] - 35s 4s/step - loss: 0.1828 - accuracy: 0.9234 - val_loss: 22.8612 - val_accuracy: 0.4967
Epoch 3/1000
8/8 [==============================] - 37s 5s/step - loss: 0.2578 - accuracy: 0.8986 - val_loss: 16.2558 - val_accuracy: 0.4967
Epoch 4/1000
8/8 [==============================] - 34s 4s/step - loss: 0.2093 - accuracy: 0.9358 - val_loss: 24.2677 - val_accuracy: 0.5033
Epoch 5/1000
8/8 [==============================] - 34s 4s/step - loss: 0.2032 - accuracy: 0.9130 - val_loss: 67.3657 - val_accuracy: 0.5033
Epoch 6/1000
8/8 [==============================] - 34s 4s/step - loss: 0.1718 - accuracy: 0.9400 - val_loss: 40.0999 - val_accuracy: 0.5033
Epoch 6: early stopping


#### 4) 성능 평가

In [ ]:
y_pred = model.predict(x_test)

4/4 [==============================] - 2s 125ms/step


In [ ]:
y_pred=model.predict(testset)

4/4 [==============================] - 5s 645ms/step


In [ ]:
y_pred=y_pred.argmax(axis=1)

In [ ]:
y_test=y_test.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
print (f'정확도는 {accuracy_score(y_test,y_pred)}입니다.')
print (confusion_matrix(y_test,y_pred))
print (classification_report(y_test,y_pred))

정확도는 0.4959349593495935입니다.
[[61  0]
 [62  0]]
              precision    recall  f1-score   support

           0       0.50      1.00      0.66        61
           1       0.00      0.00      0.00        62

    accuracy                           0.50       123
   macro avg       0.25      0.50      0.33       123
weighted avg       0.25      0.50      0.33       123



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
